In [206]:
import pandas as pd
import sys
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import math
from math import sqrt
sys.path.insert(0,"../")

In [207]:
import logging

from tqdm.notebook import tqdm
tqdm.pandas()

logger = logging.getLogger(__name__)

logger.setLevel(logging.CRITICAL)

import warnings; warnings.simplefilter('ignore')


# Read in predictions data + true close 

In [208]:
btc_predictions_df = pd.read_csv("training_configs/btc_all_predictions.csv")
true_price_df = pd.read_csv("../tmp/historic_crypto_prices - bitcoin_jan_2017_sep_4_2021 copy.csv")

In [209]:
btc_predictions_df.head()

,date,nbeats_btc_lookback_15_window_5_std_1.25_num_add_dfs_2,tcn_btc_lookback_15_window_5_std_1.25_num_add_dfs_2,nbeats_btc_lookback_30_window_5_std_1.25_num_add_dfs_2,tcn_btc_lookback_30_window_5_std_1.25_num_add_dfs_2,nbeats_btc_lookback_45_window_5_std_1.25_num_add_dfs_2,tcn_btc_lookback_45_window_5_std_1.25_num_add_dfs_2,date_prediction_for,test_model_lookback_1
0,2010-01-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2010-01-09,0.0
1,2019-04-02,4752.149527,253.537135,4730.976374,654.310204,3469.734359,795.201430,2019-04-09,0.0
2,2019-04-03,4546.248123,407.206977,4373.560445,255.627627,4437.900633,1413.509939,2019-04-10,0.0
3,2019-04-04,4550.998590,726.168584,4045.591506,1861.981408,4012.011230,523.093010,2019-04-11,0.0
4,2019-04-05,5003.264197,353.606543,4819.668698,817.215937,3977.238773,553.057301,2019-04-12,0.0


In [210]:
btc_predictions_df.tail()

,date,nbeats_btc_lookback_15_window_5_std_1.25_num_add_dfs_2,tcn_btc_lookback_15_window_5_std_1.25_num_add_dfs_2,nbeats_btc_lookback_30_window_5_std_1.25_num_add_dfs_2,tcn_btc_lookback_30_window_5_std_1.25_num_add_dfs_2,nbeats_btc_lookback_45_window_5_std_1.25_num_add_dfs_2,tcn_btc_lookback_45_window_5_std_1.25_num_add_dfs_2,date_prediction_for,test_model_lookback_1
1096,2022-04-01,47197.423190,43981.505209,45904.760226,45414.193761,44251.983524,45981.175677,2022-04-08,0.0
1097,2022-04-02,44834.376826,45088.403326,49491.837986,42798.696287,44422.556391,41990.627225,2022-04-09,0.0
1098,2022-04-03,42896.271421,45404.733748,49415.744984,44892.617557,42476.271434,41902.335856,2022-04-10,0.0
1099,2022-04-04,44852.974859,44460.093843,43443.254603,45503.008668,45533.537186,41151.918788,2022-04-11,0.0
1100,2022-04-05,48770.351444,44594.743329,47205.332004,46372.801843,50676.631747,38928.288902,2022-04-12,0.0


In [211]:
true_price_df.head()

,date,open,high,low,close,volume
0,2017-01-01,963.66,1003.08,958.70,998.33,147775008.0
1,2017-01-02,998.62,1031.39,996.70,1021.75,222184992.0
2,2017-01-03,1021.60,1044.08,1021.60,1043.84,185168000.0
3,2017-01-04,1044.40,1159.42,1044.40,1154.73,344945984.0
4,2017-01-05,1156.73,1191.10,910.42,1013.38,510199008.0


- merged true close price and prediction

In [285]:
true_price_df = true_price_df[['date','close']]
merged_df = pd.merge(btc_predictions_df, true_price_df, left_on='date_prediction_for',
                     right_on ='date', suffixes=['_pred','_true'])

In [286]:
merged_df['day'] = [t.day for t in pd.to_datetime(merged_df.date_prediction_for)]
merged_df['month'] =  [t.month for t in pd.to_datetime(merged_df.date_prediction_for)]
merged_df['quarter'] = [t.quarter for t in pd.to_datetime(merged_df.date_prediction_for)]
merged_df['day_of_year'] = [t.strftime('%j') for t in pd.to_datetime(merged_df.date_prediction_for)]
merged_df['year'] = [t.year for t in pd.to_datetime(merged_df.date_prediction_for)]

In [287]:
dropped_merged_df = merged_df.drop(['date_pred', 'date_true', 'date_prediction_for'],  axis=1)

# Test the different algos
- mean
- RF
- GB
- LR

In [288]:
dropped_merged_df

,nbeats_btc_lookback_15_window_5_std_1.25_num_add_dfs_2,tcn_btc_lookback_15_window_5_std_1.25_num_add_dfs_2,nbeats_btc_lookback_30_window_5_std_1.25_num_add_dfs_2,tcn_btc_lookback_30_window_5_std_1.25_num_add_dfs_2,nbeats_btc_lookback_45_window_5_std_1.25_num_add_dfs_2,tcn_btc_lookback_45_window_5_std_1.25_num_add_dfs_2,test_model_lookback_1,close,day,month,quarter,day_of_year,year
0,4752.149527,253.537135,4730.976374,654.310204,3469.734359,795.201430,0.0,5204.96,9,4,2,099,2019
1,4546.248123,407.206977,4373.560445,255.627627,4437.900633,1413.509939,0.0,5324.55,10,4,2,100,2019
2,4550.998590,726.168584,4045.591506,1861.981408,4012.011230,523.093010,0.0,5064.49,11,4,2,101,2019
3,5003.264197,353.606543,4819.668698,817.215937,3977.238773,553.057301,0.0,5089.54,12,4,2,102,2019
4,5184.268504,197.768508,5123.185573,1596.333494,4093.602236,920.106602,0.0,5096.59,13,4,2,103,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1089,43746.323175,42682.902673,42044.482758,40531.215250,45463.902499,41495.520013,0.0,45817.00,2,4,2,092,2022
1090,43266.796507,39898.750552,38788.181184,40255.229593,41747.739482,43083.086030,0.0,46405.00,3,4,2,093,2022
1091,44123.399870,44515.980426,43370.000462,44870.132396,44741.252847,43844.835490,0.0,46592.00,4,4,2,094,2022
1092,48795.983121,47274.369412,48410.423252,45928.678493,41806.109635,42099.064473,0.0,45504.00,5,4,2,095,2022


In [290]:
merged_df

,date_pred,nbeats_btc_lookback_15_window_5_std_1.25_num_add_dfs_2,tcn_btc_lookback_15_window_5_std_1.25_num_add_dfs_2,nbeats_btc_lookback_30_window_5_std_1.25_num_add_dfs_2,tcn_btc_lookback_30_window_5_std_1.25_num_add_dfs_2,nbeats_btc_lookback_45_window_5_std_1.25_num_add_dfs_2,tcn_btc_lookback_45_window_5_std_1.25_num_add_dfs_2,date_prediction_for,test_model_lookback_1,date_true,close,day,month,quarter,day_of_year,year
0,2019-04-02,4752.149527,253.537135,4730.976374,654.310204,3469.734359,795.201430,2019-04-09,0.0,2019-04-09,5204.96,9,4,2,099,2019
1,2019-04-03,4546.248123,407.206977,4373.560445,255.627627,4437.900633,1413.509939,2019-04-10,0.0,2019-04-10,5324.55,10,4,2,100,2019
2,2019-04-04,4550.998590,726.168584,4045.591506,1861.981408,4012.011230,523.093010,2019-04-11,0.0,2019-04-11,5064.49,11,4,2,101,2019
3,2019-04-05,5003.264197,353.606543,4819.668698,817.215937,3977.238773,553.057301,2019-04-12,0.0,2019-04-12,5089.54,12,4,2,102,2019
4,2019-04-06,5184.268504,197.768508,5123.185573,1596.333494,4093.602236,920.106602,2019-04-13,0.0,2019-04-13,5096.59,13,4,2,103,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1089,2022-03-26,43746.323175,42682.902673,42044.482758,40531.215250,45463.902499,41495.520013,2022-04-02,0.0,2022-04-02,45817.00,2,4,2,092,2022
1090,2022-03-27,43266.796507,39898.750552,38788.181184,40255.229593,41747.739482,43083.086030,2022-04-03,0.0,2022-04-03,46405.00,3,4,2,093,2022
1091,2022-03-28,44123.399870,44515.980426,43370.000462,44870.132396,44741.252847,43844.835490,2022-04-04,0.0,2022-04-04,46592.00,4,4,2,094,2022
1092,2022-03-29,48795.983121,47274.369412,48410.423252,45928.678493,41806.109635,42099.064473,2022-04-05,0.0,2022-04-05,45504.00,5,4,2,095,2022


## Compare different algos
- Test each day individually

In [ ]:
gb_pred_list = []
rf_pred_list = []
lr_pred_list = []
mean_pred_list = []
actual_close_list = []

COLS_TO_EXCLUDE = ['test_model_lookback_1', 'close','rolling_mean','bollinger_high','bollinger_low']
DAY_PART_COLS = ['day', 'month', 'quarter', 'day_of_year', 'year']

for row_idx, row in enumerate(dropped_merged_df.iterrows()):
    if row_idx == 0: # need one day to start training
        continue
    
    if row_idx % 10 == 0:
        print(f"{row_idx/len(dropped_merged_df)*100:.2f}%")

    gb = GradientBoostingRegressor(n_estimators=150)
    rf = RandomForestRegressor(n_estimators=200, n_jobs=-1)
    lr = LinearRegression()
    
    
    stacking_training_cols = row[1].index[~row[1].index.isin(COLS_TO_EXCLUDE)]
    stacked_x_data = dropped_merged_df[stacking_training_cols][:row_idx] # don't include the current row
    stacked_y_data = dropped_merged_df[PRED_COL][:row_idx] # don't include the current row
    
    gb.fit(stacked_x_data, stacked_y_data)
    gb_pred = gb.predict(np.array(dropped_merged_df[stacking_training_cols].iloc[row_idx]).reshape(1,-1))
    gb_pred_list.append(gb_pred[0])
    
    rf.fit(stacked_x_data, stacked_y_data)
    rf_pred = rf.predict(np.array(dropped_merged_df[stacking_training_cols].iloc[row_idx]).reshape(1,-1))
    rf_pred_list.append(rf_pred[0])
    
    lr.fit(stacked_x_data, stacked_y_data)
    lr_pred = lr.predict(np.array(dropped_merged_df[stacking_training_cols].iloc[row_idx]).reshape(1,-1))
    lr_pred_list.append(lr_pred[0])
    
    
    mean_pred_cols = row[1].index[~row[1].index.isin(COLS_TO_EXCLUDE+DAY_PART_COLS)]
    mean_pred = np.mean(row[1][mean_pred_cols])
    mean_pred_list.append(mean_pred)
    
    actual_close_list.append(row[1][PRED_COL])


0.91%
1.83%
2.74%
3.66%
4.57%
5.48%
6.40%
7.31%
8.23%
9.14%
10.05%
10.97%
11.88%
12.80%
13.71%
14.63%
15.54%
16.45%
17.37%
18.28%
19.20%
20.11%
21.02%
21.94%
22.85%
23.77%
24.68%
25.59%
26.51%
27.42%
28.34%
29.25%
30.16%
31.08%
31.99%
32.91%
33.82%
34.73%
35.65%
36.56%
37.48%
38.39%
39.31%
40.22%
41.13%
42.05%
42.96%
43.88%
44.79%
45.70%
46.62%
47.53%
48.45%
49.36%
50.27%
51.19%
52.10%
53.02%
53.93%
54.84%
55.76%
56.67%
57.59%
58.50%


In [ ]:
rf_pred_list[-10:]

In [ ]:
mean_pred_list[-10:]

In [ ]:
gb_pred_list[-10:]

In [ ]:
rf_pred_list[-10:]

In [ ]:
actual_close_list[-10:]

In [ ]:
dropped_merged_df["nbeats_btc_lookback_15_window_5_std_1.25_num_add_dfs_2"]

In [ ]:
rmse_nbeats_15 = sqrt(mean_squared_error(dropped_merged_df["nbeats_btc_lookback_15_window_5_std_1.25_num_add_dfs_2"], dropped_merged_df.close))
rmse_nbeats_30 = sqrt(mean_squared_error(dropped_merged_df["nbeats_btc_lookback_30_window_5_std_1.25_num_add_dfs_2"], dropped_merged_df.close))
rmse_nbeats_45 = sqrt(mean_squared_error(dropped_merged_df["nbeats_btc_lookback_45_window_5_std_1.25_num_add_dfs_2"], dropped_merged_df.close))

rmse_tcn_15 = sqrt(mean_squared_error(dropped_merged_df["tcn_btc_lookback_15_window_5_std_1.25_num_add_dfs_2"], dropped_merged_df.close))
rmse_tcn_30 = sqrt(mean_squared_error(dropped_merged_df["tcn_btc_lookback_30_window_5_std_1.25_num_add_dfs_2"], dropped_merged_df.close))
rmse_tcn_45 = sqrt(mean_squared_error(dropped_merged_df["tcn_btc_lookback_45_window_5_std_1.25_num_add_dfs_2"], dropped_merged_df.close))

In [ ]:
rmse_gb = sqrt(mean_squared_error(gb_pred_list, actual_close_list))
rmse_rf = sqrt(mean_squared_error(rf_pred_list, actual_close_list))
rmse_lr = sqrt(mean_squared_error(lr_pred_list, actual_close_list))
rmse_average = sqrt(mean_squared_error(mean_pred_list, actual_close_list))

print(f"rmse_gb = {rmse_gb}")
print(f"rmse_rf = {rmse_rf}")
print(f"rmse_lr = {rmse_lr}")
print(f"rmse_average = {rmse_average}")
print(f" rmse_nbeats_15 rmse ={rmse_nbeats_15}")
print(f" rmse_nbeats_30 rmse ={rmse_nbeats_30}")
print(f" rmse_nbeats_45 rmse ={rmse_nbeats_45}")
print(f" rmse_tcn_15 rmse ={rmse_tcn_15}")
print(f" rmse_tcn_30 rmse ={rmse_tcn_30}")
print(f" rmse_tcn_45 rmse ={rmse_tcn_45}")

In [ ]:
rmse_rf/rmse_gb-1 # 5% better rmse = 2048 RF w/o year, 1965 with year , 4% better with year

In [ ]:
rmse_rf/rmse_average-1 # lol 52% better

# Test determine trading state strategy


In [336]:
def _determine_win_or_loss_amount(row, mode, buy_entry_price, todays_close_price, position_entry_date, 
                                  n_buy_lost, n_buy_won, dollar_amount_buy_won, dollar_amount_buy_lost):
       #self._determine_win_or_loss_amount(row)
    # lost money
    if mode == "buy" and buy_entry_price > todays_close_price:

        lost_amount = buy_entry_price - todays_close_price
        print(f"Lost {lost_amount} on this trade")

        n_buy_lost += 1
        dollar_amount_buy_lost += lost_amount

    # made money
    elif mode == "buy" and buy_entry_price < todays_close_price:

        won_amount = todays_close_price - buy_entry_price
        print(f"Won {won_amount} on this trade")
        n_buy_won += 1
        dollar_amount_buy_won += won_amount
    else:
        raise ValueError("Something went wrong calculating win/lose amount")

    days_in_trade = pd.to_datetime(row.name) - pd.to_datetime(position_entry_date)
    logger.info(f"days in trades = {days_in_trade.days}")
    n_total_days_in_trades += days_in_trade.days[0]

    # info logging

    print(
        f"Average days in trades = {n_total_days_in_trades/(n_buy_won  + n_buy_lost )}"
    )
    if n_buy_won > 0 or n_buy_lost > 0:
        print(f"Bat rate buy so far = {n_buy_won / (n_buy_won + n_buy_lost)}")


    if dollar_amount_buy_won > 0 or dollar_amount_buy_lost > 0:
        logger.info(
            f"Win rate buy so far = {float(dollar_amount_buy_won) / float(float(dollar_amount_buy_won) + float(dollar_amount_buy_lost))}"
        )


    print(f"Total days in trades = {n_total_days_in_trades }")

    # reset for sanity
    position_entry_date = None
        
    return mode, buy_entry_price, todays_close_price, position_entry_date, n_buy_lost, n_buy_won, dollar_amount_buy_won, dollar_amount_buy_lost
        

In [337]:
merged_df.index = merged_df.date_true
dropped_merged_df.index=merged_df.index

In [340]:
dropped_merged_df

,nbeats_btc_lookback_15_window_5_std_1.25_num_add_dfs_2,tcn_btc_lookback_15_window_5_std_1.25_num_add_dfs_2,nbeats_btc_lookback_30_window_5_std_1.25_num_add_dfs_2,tcn_btc_lookback_30_window_5_std_1.25_num_add_dfs_2,nbeats_btc_lookback_45_window_5_std_1.25_num_add_dfs_2,tcn_btc_lookback_45_window_5_std_1.25_num_add_dfs_2,test_model_lookback_1,close,day,month,quarter,day_of_year,year,rolling_mean,bollinger_high,bollinger_low
date_true,,,,,,,,,,,,,,,,
2019-04-09,4752.149527,253.537135,4730.976374,654.310204,3469.734359,795.201430,0.0,5204.96,9,4,2,099,2019,NaN,NaN,NaN
2019-04-10,4546.248123,407.206977,4373.560445,255.627627,4437.900633,1413.509939,0.0,5324.55,10,4,2,100,2019,NaN,NaN,NaN
2019-04-11,4550.998590,726.168584,4045.591506,1861.981408,4012.011230,523.093010,0.0,5064.49,11,4,2,101,2019,NaN,NaN,NaN
2019-04-12,5003.264197,353.606543,4819.668698,817.215937,3977.238773,553.057301,0.0,5089.54,12,4,2,102,2019,NaN,NaN,NaN
2019-04-13,5184.268504,197.768508,5123.185573,1596.333494,4093.602236,920.106602,0.0,5096.59,13,4,2,103,2019,5156.026,5291.705339,5020.346661
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-04-02,43746.323175,42682.902673,42044.482758,40531.215250,45463.902499,41495.520013,0.0,45817.00,2,4,2,092,2022,46435.600,47463.019629,45408.180371
2022-04-03,43266.796507,39898.750552,38788.181184,40255.229593,41747.739482,43083.086030,0.0,46405.00,3,4,2,093,2022,46225.400,46975.689840,45475.110160
2022-04-04,44123.399870,44515.980426,43370.000462,44870.132396,44741.252847,43844.835490,0.0,46592.00,4,4,2,094,2022,46126.800,46681.603879,45571.996121


In [351]:
strategy = 'mean' # mean, RF, GB
verbose = True

mode = 'no_position' # no_position, buy
action_to_take = 'none_to_none' # none_to_none, none_to_buy, buy_to_continue_buy, buy_to_none
has_crossed_mean = False
buy_entry_price = 0
stop_loss_price = 0

PRED_COL = 'close'
window = 5
no_of_std = 1.25
stop_loss_pct=0.10
rolling_mean_col = 'rolling_mean'
bollinger_high_col = 'bollinger_high'
bollinger_low_col = 'bollinger_low'

dollar_amount_buy_lost =  0.0
dollar_amount_buy_won = 0.0
n_buy_lost =  0.0
n_buy_won= 0.0
n_total_days_in_trades = 0.0

buy_has_crossed_mean = 0.0


rolling_mean = dropped_merged_df[PRED_COL].rolling(window).mean()
rolling_std = dropped_merged_df[PRED_COL].rolling(window).std()

dropped_merged_df[rolling_mean_col] = rolling_mean
dropped_merged_df[bollinger_high_col] = rolling_mean + (rolling_std * no_of_std)
dropped_merged_df[bollinger_low_col] = rolling_mean - (rolling_std * no_of_std)
dropped_merged_df.fillna(method='bfill')


# class vars
price_predictions = 0.0
position_entry_date = None



for row_idx, row in enumerate(dropped_merged_df.iterrows()):
    if row_idx == 0: # need one day to start training
        continue
    
    if row_idx % 10 == 0:
        print(f"{row_idx/len(dropped_merged_df)*100:.2f}%")
        
    # current stats
    row = dropped_merged_df.iloc[row_idx]
    prev_row = dropped_merged_df.iloc[row_idx-1:row_idx]
    todays_close_price = row[PRED_COL]
    close_col = PRED_COL
    todays_rolling_mean = row[rolling_mean_col]
    todays_bollinger_high = row[bollinger_high_col]
    todays_bollinger_low = row[bollinger_low_col]
    previous_days_close = prev_row[close_col].values[0]
    previous_days_bollinger_low = prev_row[bollinger_low_col].values[0]
    
    if verbose:
        print(todays_bollinger_high,' todays_bollinger_high')
        print(todays_close_price, 'todays_close_price')
        print(todays_bollinger_low, 'todays_bollinger_low')
        print(previous_days_close,' ,previous_days_close')
        print(previous_days_bollinger_low, 'previous_days_bollinger_low')
        print(mode,' mode')
        print(row.index)
        print(stop_loss_price, ' stop_loss_price')
        
    # price prediction
    
    if strategy == 'mean':
        mean_pred_cols = row.index[~row.index.isin(COLS_TO_EXCLUDE+DAY_PART_COLS)]
        if verbose:
            print(mean_pred_cols, 'mean_pred_cols')
            print(row[mean_pred_cols], 'row[mean_pred_cols]')
        price_prediction = np.mean(row[mean_pred_cols])
        print(price_prediction, 'price_prediction')
        
        
    
    

    
    # update stop loss
    print(mode, 'mode')
    if mode == "buy" and (1 - stop_loss_pct) * todays_close_price > stop_loss_price:
        stop_loss_price = (1 - stop_loss_pct) * todays_close_price
        print(f"Updating stop loss to {stop_loss_price}", row)
        
        
    # check if we've previously crossed the mean trailing price
    if mode == "buy" and todays_close_price > todays_rolling_mean:
        buy_has_crossed_mean = 1
        
        # stop loss, get out of buy position
        logger.info(f"self.stop_loss_price = {stop_loss_price}")
    if mode == "buy" and (stop_loss_price > todays_close_price):
        print("stop loss activated for getting out of our buy", row)

        # determine win or loss
        print('determining win or loss')
        mode, buy_entry_price, todays_close_price, position_entry_date, n_buy_lost, n_buy_won, dollar_amount_buy_won, dollar_amount_buy_lost = _determine_win_or_loss_amount(row,  mode, buy_entry_price, todays_close_price, position_entry_date, 
                                  n_buy_lost, n_buy_won, dollar_amount_buy_won, dollar_amount_buy_lost)
        
        
        
        
        
        # record keeping

        mode = "no_position"
        action_to_take = "buy_to_none"
        buy_has_crossed_mean = 0
        buy_entry_price = 0
        stop_loss_price = 0
        
        # buy -> no_position? no position is below running mean
        # or, if we are above the top band (mean reversion)
    elif mode == "buy" and (
            (
                todays_close_price < todays_rolling_mean
                and buy_has_crossed_mean == 1
            )
            or (todays_close_price > todays_bollinger_high)
            or (todays_close_price < todays_bollinger_low)
            or (todays_rolling_mean < buy_entry_price)
        ):
            
        ## check buy to none
        if (
            (price_prediction < todays_rolling_mean)
            or (price_prediction < buy_entry_price)
            or (todays_rolling_mean < buy_entry_price)
        ):
            # note the order here matters
            # determine win or loss
            print('determining win or loss')
            mode, buy_entry_price, todays_close_price, position_entry_date, n_buy_lost, n_buy_won, dollar_amount_buy_won, dollar_amount_buy_lost = _determine_win_or_loss_amount(row,  mode, buy_entry_price, todays_close_price, position_entry_date, 
                              n_buy_lost, n_buy_won, dollar_amount_buy_won, dollar_amount_buy_lost)
            # record keeping

            mode = "no_position"
            action_to_take = "buy_to_none"
            buy_has_crossed_mean = 0
            buy_entry_price = 0
            stop_loss_price = 0

            print(f"Exiting buy position . Mode = buy_to_none {row}")
        else:
            print(f"Not exiting buy position {row}")
            action_to_take = "buy_to_continue_buy"
                
                
                
    # buy check with ML model
    elif (
        mode == "no_position"
        and todays_close_price < todays_bollinger_low
        and previous_days_close > previous_days_bollinger_low
    ):
        # check if we should buy

        #self._check_if_we_should_buy(row)

        if price_prediction > todays_rolling_mean:
            # note the order here matters. Want to write logs reflecting the current state
            mode = "buy"
            action_to_take = "none_to_buy"
            buy_entry_price = todays_close_price
            stop_loss_price = todays_close_price * (1 - stop_loss_pct)
            print(row.name, "row.index[0]")
            position_entry_date = pd.to_datetime(row.name).strftime("%Y-%m-%d")

            print(f"ml pred higher than mean taking position {row},price_prediction = {price_prediction} ")
        else:
            print(
                f"self.price_prediction is not higher than the {strategy}. Not going to buy {row}. Price prediction = {price_prediction}"
            )

            action_to_take = "none_to_none"

    elif action_to_take == "none_to_buy":  # not in a position, continue holidng
        action_to_take = "buy_to_continue_buy"
        print(
            f"Taking no action today. Updating none_to_buy to buy_to_continue_buy { row}"
        )

    else:
        if action_to_take == "buy_to_none":
            action_to_take = "none_to_none"
            # self._write_and_print_log_statements(
            #     "Taking no action today. Hit the else statement and updated buy_to_none to none_to_none", row
            # )
        # else:
        #     self._write_and_print_log_statements("Taking no action today. Hit the else statement", row)


nan  todays_bollinger_high
5324.55 todays_close_price
nan todays_bollinger_low
5204.96  ,previous_days_close
nan previous_days_bollinger_low
no_position  mode
Index(['nbeats_btc_lookback_15_window_5_std_1.25_num_add_dfs_2',
       'tcn_btc_lookback_15_window_5_std_1.25_num_add_dfs_2',
       'nbeats_btc_lookback_30_window_5_std_1.25_num_add_dfs_2',
       'tcn_btc_lookback_30_window_5_std_1.25_num_add_dfs_2',
       'nbeats_btc_lookback_45_window_5_std_1.25_num_add_dfs_2',
       'tcn_btc_lookback_45_window_5_std_1.25_num_add_dfs_2',
       'test_model_lookback_1', 'close', 'day', 'month', 'quarter',
       'day_of_year', 'year', 'rolling_mean', 'bollinger_high',
       'bollinger_low'],
      dtype='object')
0  stop_loss_price
Index(['nbeats_btc_lookback_15_window_5_std_1.25_num_add_dfs_2',
       'tcn_btc_lookback_15_window_5_std_1.25_num_add_dfs_2',
       'nbeats_btc_lookback_30_window_5_std_1.25_num_add_dfs_2',
       'tcn_btc_lookback_30_window_5_std_1.25_num_add_dfs_2',
       '

UnboundLocalError: local variable 'n_total_days_in_trades' referenced before assignment

In [306]:
dropped_merged_df[10-1:10]

,nbeats_btc_lookback_15_window_5_std_1.25_num_add_dfs_2,tcn_btc_lookback_15_window_5_std_1.25_num_add_dfs_2,nbeats_btc_lookback_30_window_5_std_1.25_num_add_dfs_2,tcn_btc_lookback_30_window_5_std_1.25_num_add_dfs_2,nbeats_btc_lookback_45_window_5_std_1.25_num_add_dfs_2,tcn_btc_lookback_45_window_5_std_1.25_num_add_dfs_2,test_model_lookback_1,close,day,month,quarter,day_of_year,year,rolling_mean,bollinger_high,bollinger_low
9,5418.172138,2489.199356,4117.576533,581.4992,3261.251466,1370.090359,0.0,5298.39,18,4,2,108,2019,5204.144,5316.398517,5091.889483
